# How likely is a Bones Day vs. a No Bones Day?

In [1]:
%load_ext lab_black

### Load Python tools

In [2]:
import pandas as pd
import altair as alt
import datetime as dt

In [3]:
today = dt.datetime.today()

### Read data collected from [TikTok](https://www.tiktok.com/@jongraz) about Noodle

In [4]:
src = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vQz8eEafXnIeKsNvGL8M56Ia6vu89JDCjfJ22ORl65So4kRYtmTbwmqLjE223fe4mjcZqkrs-KT6j8z/pub?\
gid=0&single=true&output=csv"
)

### Process dates

In [5]:
src["date"] = pd.to_datetime(src["date"])

In [6]:
df_date = pd.DataFrame()
index = pd.date_range(start="2021-09-01", end=today, freq="D")
df_date["date"] = index

### Merge the dates to backfill

In [7]:
df = pd.merge(src, df_date, on="date", how="outer")

In [8]:
df["weekday"] = df["date"].dt.day_name()
df["month"] = df["date"].dt.month_name()
df["month-year"] = df["date"].dt.strftime("%Y-%m")
df["weekend"] = df["date"].dt.day_name().isin(["Saturday", "Sunday"])

In [9]:
# df.drop(["url"], axis=1, inplace=True)

In [10]:
df.bones_nobones = df.bones_nobones.fillna("No video")

### What's our dataframe look like? 

In [11]:
df.head()

,date,bones_nobones,url,weekday,month,month-year,weekend
0,2022-01-17,No bones,https://www.tiktok.com/@jongraz/video/70538379...,Monday,January,2022-01,False
1,2022-01-15,Bones!,https://vm.tiktok.com/TTPdrYXuSY/,Saturday,January,2022-01,True
2,2022-01-13,Bones!,https://www.tiktok.com/@jongraz/video/70523416...,Thursday,January,2022-01,False
3,2022-01-11,No bones,https://www.tiktok.com/@jongraz/video/70519729...,Tuesday,January,2022-01,False
4,2022-01-07,Bones!,https://www.tiktok.com/@jongraz/video/70504911...,Friday,January,2022-01,False


### Percentage of days with Bones (only days with a video)

In [12]:
(
    df[df["bones_nobones"] != "No video"].value_counts("bones_nobones", normalize=True)
    * 100
).round(0)

bones_nobones
Bones!      51.0
No bones    49.0
dtype: float64

### Group by day type

In [13]:
days = (
    df.groupby(["bones_nobones"])
    .agg({"date": "count"})
    .reset_index()
    .rename(columns={"date": "count"})
)

### Simple bar chart explaining the count of day types

In [14]:
alt.Chart(days[days["bones_nobones"] != "No video"], title=" ").mark_bar().encode(
    y=alt.Y("bones_nobones", title="", axis=alt.Axis(tickCount=0, tickColor="#ffffff")),
    x=alt.X(
        "count",
        title="Days",
        axis=alt.Axis(grid=False, tickCount=6, tickColor="#ffffff"),
    ),
    color=alt.Color(
        "bones_nobones",
        title="",
        legend=None,
        scale=alt.Scale(
            domain=["Bones!", "No bones", "No video"],
            range=["#5ab4ac", "#d8b365", "#e6e6e6"],
        ),
    ),
).properties(width=500, height=70)

alt.Chart(...)

### Plot Noodle's days on a calendar

In [15]:
heatmap = (
    alt.Chart(df[df["date"] > "2021-08-31"], title=" ")
    .mark_rect()
    .encode(
        x=alt.X("date(date):O", title=" "),
        y=alt.Y("month(date):O", title=""),
        color=alt.Color(
            "bones_nobones",
            title="",
            scale=alt.Scale(
                domain=["Bones!", "No bones", "No video"],
                range=["#5ab4ac", "#d8b365", "#e6e6e6"],
            ),
        ),
        tooltip=[
            alt.Tooltip("monthdate(date):T", title="Date"),
            alt.Tooltip("bones_nobones", title="Tweets"),
        ],
    )
    .properties(width=500, height=70, title="Bones days since Sept. 1")
)

heatmap.configure_legend(orient="top", padding=10).configure_view(strokeOpacity=0)

alt.Chart(...)

### Prep for datawrapper 

In [16]:
df.head()

,date,bones_nobones,url,weekday,month,month-year,weekend
0,2022-01-17,No bones,https://www.tiktok.com/@jongraz/video/70538379...,Monday,January,2022-01,False
1,2022-01-15,Bones!,https://vm.tiktok.com/TTPdrYXuSY/,Saturday,January,2022-01,True
2,2022-01-13,Bones!,https://www.tiktok.com/@jongraz/video/70523416...,Thursday,January,2022-01,False
3,2022-01-11,No bones,https://www.tiktok.com/@jongraz/video/70519729...,Tuesday,January,2022-01,False
4,2022-01-07,Bones!,https://www.tiktok.com/@jongraz/video/70504911...,Friday,January,2022-01,False


In [17]:
df["bones_class"] = (
    df["bones_nobones"]
    .str.lower()
    .str.replace(" ", "-", regex=False)
    .str.replace("!", "", regex=False)
)

In [18]:
df["weekend_class"] = (
    df["weekend"]
    .astype(str)
    .str.lower()
    .str.replace(" ", "-", regex=False)
    .str.replace("!", "", regex=False)
)

In [19]:
grouped_all = (
    df[df["date"] > "2021-09-30"]
    .groupby("bones_nobones")
    .agg({"date": "size"})
    .reset_index()
    .rename(columns={"date": "Days"})
)

In [20]:
grouped_all

,bones_nobones,Days
0,Bones!,37
1,No bones,30
2,No video,42


In [21]:
grouped_all.iloc[2, 1]

42

In [22]:
grouped_weekends = (
    df[df["date"] > "2021-09-30"]
    .groupby(["bones_nobones", "weekend"])
    .agg({"date": "size"})
    .reset_index()
    .rename(columns={"date": "Days"})
)

In [23]:
grouped_weekends

,bones_nobones,weekend,Days
0,Bones!,False,29
1,Bones!,True,8
2,No bones,False,24
3,No bones,True,6
4,No video,False,24
5,No video,True,18


In [24]:
grouped_weekends.iloc[4, 2]

24

In [25]:
stacked_bar = pd.DataFrame(
    {
        "Category": ["All days", "Weekends", "Weekdays"],
        "Bones": [
            grouped_all.iloc[0, 1],
            grouped_weekends.iloc[1, 2],
            grouped_weekends.iloc[0, 2],
        ],
        "No Bones": [
            grouped_all.iloc[1, 1],
            grouped_weekends.iloc[3, 2],
            grouped_weekends.iloc[2, 2],
        ],
        "No Video": [
            grouped_all.iloc[2, 1],
            grouped_weekends.iloc[5, 2],
            grouped_weekends.iloc[4, 2],
        ],
    }
)

In [26]:
# stacked_bar["bones"] = stacked_bar["Bones"]
# stacked_bar["nobones"] = stacked_bar["No Bones"]

In [27]:
stacked_bar

,Category,Bones,No Bones,No Video
0,All days,37,30,42
1,Weekends,8,6,18
2,Weekdays,29,24,24


---

In [28]:
df["week"] = pd.to_datetime(df["date"]).dt.isocalendar().week
df["weekstart"] = pd.to_datetime(df["date"]) - pd.to_timedelta(
    pd.to_datetime(df["date"]).dt.dayofweek, unit="d"
)

In [29]:
df.head()

,date,bones_nobones,url,weekday,month,month-year,weekend,bones_class,weekend_class,week,weekstart
0,2022-01-17,No bones,https://www.tiktok.com/@jongraz/video/70538379...,Monday,January,2022-01,False,no-bones,false,3,2022-01-17
1,2022-01-15,Bones!,https://vm.tiktok.com/TTPdrYXuSY/,Saturday,January,2022-01,True,bones,true,2,2022-01-10
2,2022-01-13,Bones!,https://www.tiktok.com/@jongraz/video/70523416...,Thursday,January,2022-01,False,bones,false,2,2022-01-10
3,2022-01-11,No bones,https://www.tiktok.com/@jongraz/video/70519729...,Tuesday,January,2022-01,False,no-bones,false,2,2022-01-10
4,2022-01-07,Bones!,https://www.tiktok.com/@jongraz/video/70504911...,Friday,January,2022-01,False,bones,false,1,2022-01-03


In [30]:
weeks = (
    df.groupby(["weekstart", "bones_class"])
    .agg({"date": "count"})
    .reset_index()
    .rename(columns={"date": "count", "bones_class": "result"})
)

In [31]:
weeks.head()

,weekstart,result,count
0,2021-03-08,no-bones,1
1,2021-08-09,no-bones,1
2,2021-08-16,no-bones,1
3,2021-08-30,no-bones,1
4,2021-08-30,no-video,4


In [32]:
weeks_pivot = pd.pivot_table(
    weeks, values="count", index="weekstart", columns="result", fill_value=0
).reset_index()

In [33]:
weeks_pivot.to_csv("../_data/bonesdays_weeks.csv", index=False)
weeks_pivot.to_csv("../assets/data/bonesdays_weeks.csv", index=False)

---

In [34]:
months = (
    df[df["date"] > "2021-09-30"]
    .groupby(["month", "month-year", "bones_class"])
    .agg({"date": "count"})
    .reset_index()
    .rename(columns={"date": "count", "bones_class": "result"})
)

In [35]:
months

,month,month-year,result,count
0,December,2021-12,bones,10
1,December,2021-12,no-bones,8
2,December,2021-12,no-video,13
3,January,2022-01,bones,4
4,January,2022-01,no-bones,3
5,January,2022-01,no-video,10
6,November,2021-11,bones,12
7,November,2021-11,no-bones,10
8,November,2021-11,no-video,8
9,October,2021-10,bones,11


In [36]:
months_pivot = pd.pivot_table(
    months, values="count", index="month-year", columns="result", fill_value=0
).reset_index()

In [37]:
months_pivot

result,month-year,bones,no-bones,no-video
0,2021-10,11,9,11
1,2021-11,12,10,8
2,2021-12,10,8,13
3,2022-01,4,3,10


In [38]:
months_pivot.to_csv("../_data/bonesdays_months.csv", index=False)
months_pivot.to_csv("../assets/data/bonesdays_months.csv", index=False)

---

### Export

In [39]:
df["display_date"] = df["date"].dt.strftime("%b. %-d ")
df["date"] = df["date"].astype(str)

In [40]:
stacked_bar.to_csv("../_data/bonesdays_summary_csv.csv", index=False)
stacked_bar.to_csv("../assets/data/bonesdays_summary.csv", index=False)

In [41]:
df[df["bones_nobones"] != "No video"].to_csv("../_data/bonesdays_csv.csv", index=False)
df[df["bones_nobones"] != "No video"].to_csv(
    "../assets/data/bonesdays.csv", index=False
)

In [42]:
stacked_bar.to_json("../_data/bonesdays_summary.json", indent=4, orient="records")
stacked_bar.to_json("../assets/data/bonesdays_summary.json", indent=4, orient="records")

In [43]:
df[df["bones_nobones"] != "No video"].to_json(
    "../_data/bonesdays.json", indent=4, orient="records"
)
df[df["bones_nobones"] != "No video"].to_json(
    "../assets/data/bonesdays.json", indent=4, orient="records"
)

In [44]:
today = dt.datetime.today().strftime("%Y-%m-%d")
display_today = dt.datetime.today().strftime("%b. %-d")

In [45]:
toplines = pd.DataFrame(
    {
        "bones": [grouped_all.iloc[0, 1]],
        "nobones": [grouped_all.iloc[1, 1]],
        "novideo": [grouped_all.iloc[2, 1]],
    }
)

In [46]:
toplines["updated_date"] = today

In [47]:
toplines

,bones,nobones,novideo,updated_date
0,37,30,42,2022-01-16


In [48]:
toplines["display_date"] = display_today

In [49]:
toplines.to_json("../_data/toplines.json", indent=4, orient="records")
toplines.to_json("../assets/data/toplines.json", indent=4, orient="records")